In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from matplotlib import style
import math
import random


In [107]:
#LP MAXIMIZATION MODEL

from pulp import *
Promo_Model = LpProblem("Pharma_Promotion_Model", LpMaximize)


In [108]:
#PARAMETERS

T=30 # end of promotion horizon
PHorizon = list(range(0, T))

I=10 # product count 
Products = list(range(0, I))

#NOT USED RIGHT NOW
Promo_Levels = [0,1,2]

Base_Demand = np.zeros((I, T), dtype = int)

for i in Products:
    for t in PHorizon:
        Base_Demand[i][t] = random.uniform(4000,6000)
    
Promo_Sensitivity = np.zeros(I, dtype = float)
for i in Products:
    Promo_Sensitivity[i] = random.uniform(6,10)
    
Promo_Sensitivity[0] = 1
Promo_Sensitivity[I-1] = 10


Total_PaidQty_Limit = 1000000
Total_FreeQty_Limit = 250000

Promo_Sensitivity


array([ 1.        ,  7.9533866 ,  6.35054765,  7.24450098,  8.64909589,
        6.96332305,  7.74080694,  9.7351889 ,  9.00555674, 10.        ])

In [109]:
#DECISION VARIABLES

#Paid Quantity
PaidQtyLimit = 100000
P = LpVariable.dicts("Paid_Quantity",(Products,PHorizon),lowBound=0, upBound=PaidQtyLimit, cat='Integer')

#Free Quantity
FreeQtyLimit = 5000
F = LpVariable.dicts("Free_Quantity",(Products,PHorizon),lowBound=0, upBound=FreeQtyLimit, cat='Integer')

#Is Product "i" at Time "t" Promoted?
X = LpVariable.dicts("IsPromoted",(Products,PHorizon),cat='Binary')

#Promo Ratio
#R = LpVariable.dicts("Promo_Ratio",(Products,PHorizon),lowBound=0, upBound=1, cat='Continuous')


In [110]:
#OBJECTIVE FUNCTION

Total_Paid_Quantity = lpSum(lpSum(P[i][t] for i in Products) for t in PHorizon)
Total_Free_Quantity = lpSum(lpSum(F[i][t] for i in Products) for t in PHorizon)

W1 = 100 #average earning from a paid quantity
W2 = 50 #average cost for a given free quantity

Promo_Model += W1*Total_Paid_Quantity - W2*Total_Free_Quantity


In [111]:
#DEMAND SATISFACTION
for i in Products:
    for t in PHorizon:
        Promo_Model += P[i][t] + F[i][t] <= Base_Demand[i][t] + Promo_Sensitivity[i]*F[i][t]
                                                                                                
#NO FREE QTY WITHOUT PROMOTION
M = FreeQtyLimit
for i in Products:
    for t in PHorizon:
        Promo_Model += F[i][t] <= X[i][t]*M
        Promo_Model += F[i][t] >= X[i][t]-0.5

#PROMOTED PRODUCTS AT TIME "t"
Promoted_Products = 8
for t in PHorizon:
    Promo_Model += lpSum(X[i][t] for i in Products) <= Promoted_Products
    
#PROMO PERIOD FOR PRODUCT "i"  
Max_Promo_Duration = 15
Min_Promo_Duration = 10
for i in Products:
    Promo_Model += lpSum(X[i][t] for t in PHorizon) <= Max_Promo_Duration
    Promo_Model += lpSum(X[i][t] for t in PHorizon) >= Min_Promo_Duration

#TOTAL PAID QUANTITY PER PRODUCT
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(P[i][t] for t in PHorizon) <= Total_PaidQty_Limit
    
#TOTAL FREE QUANTITY PER PRODUCT
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(F[i][t] for t in PHorizon) <= Total_FreeQty_Limit


In [112]:
solver = GUROBI()
solver.solve(Promo_Model)

# The status of the solution is printed to the screen
print("Status:", LpStatus[Promo_Model.status])


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1550 rows, 900 columns and 20670 nonzeros
Model fingerprint: 0xa6da0a34
Variable types: 0 continuous, 900 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [5e+01, 1e+02]
  Bounds range     [1e+00, 1e+05]
  RHS range        [5e-01, 1e+06]
Found heuristic solution: objective 1.494771e+08
Presolve removed 772 rows and 120 columns
Presolve time: 0.01s
Presolved: 778 rows, 780 columns, 2580 nonzeros
Found heuristic solution: objective 1.858546e+08
Variable types: 0 continuous, 780 integer (300 binary)
Found heuristic solution: objective 1.859106e+08

Root relaxation: objective 6.004885e+08, 612 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

In [113]:
# OUTPUT

# OPTIMIZED OBJECTIVE FUNCTION
print("Optimal Total Paid Quantity = ", pulp.value(Promo_Model.objective),"\n")

# PRINTS VARIABLES AND OPTIMAL VALUES
for v in Promo_Model.variables():
    print(v.name, "=", v.varValue)

Optimal Total Paid Quantity =  600482100.0 

Free_Quantity_0_0 = 0.0
Free_Quantity_0_1 = 0.0
Free_Quantity_0_10 = 1.0
Free_Quantity_0_11 = 0.0
Free_Quantity_0_12 = 1.0
Free_Quantity_0_13 = 0.0
Free_Quantity_0_14 = 0.0
Free_Quantity_0_15 = 0.0
Free_Quantity_0_16 = 0.0
Free_Quantity_0_17 = 1.0
Free_Quantity_0_18 = 1.0
Free_Quantity_0_19 = 0.0
Free_Quantity_0_2 = 0.0
Free_Quantity_0_20 = 1.0
Free_Quantity_0_21 = 1.0
Free_Quantity_0_22 = 0.0
Free_Quantity_0_23 = 0.0
Free_Quantity_0_24 = 0.0
Free_Quantity_0_25 = 1.0
Free_Quantity_0_26 = 0.0
Free_Quantity_0_27 = 0.0
Free_Quantity_0_28 = 0.0
Free_Quantity_0_29 = 0.0
Free_Quantity_0_3 = 0.0
Free_Quantity_0_4 = 0.0
Free_Quantity_0_5 = 1.0
Free_Quantity_0_6 = 1.0
Free_Quantity_0_7 = 1.0
Free_Quantity_0_8 = 0.0
Free_Quantity_0_9 = 0.0
Free_Quantity_1_0 = 5000.0
Free_Quantity_1_1 = 0.0
Free_Quantity_1_10 = 0.0
Free_Quantity_1_11 = 0.0
Free_Quantity_1_12 = 0.0
Free_Quantity_1_13 = 0.0
Free_Quantity_1_14 = 0.0
Free_Quantity_1_15 = 0.0
Free_Quantity_

In [114]:
#PROMO RATIO IS CALCULATED BASED ON THE OPTIMUM P[i][t] AND F[i][t] VALUES
Promo_Ratio = np.zeros((I, T), dtype = float)

for i in Products:
    for t in PHorizon:
        Promo_Ratio[i][t] = (F[i][t].varValue)/(P[i][t].varValue + F[i][t].varValue)

df_Promo_Ratio = pd.DataFrame(Promo_Ratio,
                 index=range(1,I+1),
                 columns=range(1,T+1))
df_Promo_Ratio

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000240,0.000186,0.000233,0.000000,0.000000,...,0.000224,0.000186,0.000000,0.000000,0.000000,0.000207,0.000000,0.000000,0.000000,0.000000
2,0.113740,0.000000,0.000000,0.000000,0.110322,0.110617,0.000000,0.000000,0.109704,0.114106,...,0.000000,0.110595,0.000000,0.109635,0.113991,0.000000,0.114166,0.110190,0.109912,0.000000
3,0.000000,0.135018,0.133833,0.000000,0.138095,0.000000,0.137163,0.139117,0.133255,0.000000,...,0.133962,0.136649,0.000000,0.000000,0.136400,0.000000,0.000000,0.135693,0.139747,0.000000
4,0.121109,0.118881,0.000000,0.000000,0.122615,0.121418,0.119258,0.000000,0.122889,0.119434,...,0.000000,0.000000,0.000000,0.119184,0.000000,0.000000,0.123686,0.000000,0.000000,0.122856
5,0.104741,0.000000,0.102145,0.000000,0.102014,0.000000,0.103488,0.000000,0.102650,0.103334,...,0.000000,0.105724,0.000000,0.105150,0.102099,0.000000,0.103462,0.105666,0.000000,0.101742
6,0.128254,0.126878,0.127681,0.128452,0.000000,0.000000,0.122802,0.126228,0.000000,0.000000,...,0.127165,0.000000,0.124375,0.123594,0.000000,0.000000,0.128297,0.000000,0.000000,0.127694
7,0.115099,0.000000,0.000000,0.112186,0.000000,0.000000,0.113345,0.000000,0.114993,0.115191,...,0.000000,0.000000,0.113314,0.000000,0.114048,0.000000,0.000000,0.000000,0.113181,0.112115
8,0.093971,0.000000,0.094439,0.000000,0.094623,0.094236,0.093421,0.000000,0.091493,0.000000,...,0.094911,0.094591,0.094158,0.092113,0.092622,0.000000,0.000000,0.000000,0.094372,0.000000
9,0.100428,0.000000,0.000000,0.099663,0.101243,0.100404,0.000000,0.101084,0.000000,0.000000,...,0.000000,0.098834,0.000000,0.101102,0.101190,0.000000,0.098137,0.099265,0.101084,0.000000
10,0.092069,0.092404,0.091238,0.000000,0.000000,0.000000,0.000000,0.000000,0.090980,0.000000,...,0.091383,0.091154,0.000000,0.091746,0.091562,0.000000,0.089944,0.000000,0.092343,0.092490


In [115]:
Promo_Sensitivity


array([ 1.        ,  7.9533866 ,  6.35054765,  7.24450098,  8.64909589,
        6.96332305,  7.74080694,  9.7351889 ,  9.00555674, 10.        ])